# Sonification of Bleeding with Bank of Filters (Nature)

# First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [1]:
# imports
import os
import copy
import threading
import matplotlib
import ipywidgets
import scipy.interpolate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, Layout


## Load Data and Create Filter-Bank Signals

In [2]:
# data
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
#df = pd.read_csv('study_step0.csv', na_values=['no info', '.'], delimiter=',')

df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

delta_min = delta.min()
delta_max = delta.max()

volume_min = volume.min()
volume_max = volume.max()

#print("dataset loaded:")
#print(f"  delta:   min={delta_min:8}, max={delta_max:8.3}")
#print(f"  volume:  min={volume_min:8}, max={volume_max:8}")

## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [3]:
# sc3nb
import sc3nb as scn
import time
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang...
Done.
Registering UDP callback...
Done.
Sclang started on non default port: 57122
Booting server...
Done.
-> sc3nb started


In [4]:
#%sc FreqScope(400, 300)
#%sc s.makeGui
#%sc s.scope
#%sc s.plotTree

## Load Buffers and SynthDef

In [5]:
#assign buffers and bufnums
buffers = [
sc.Buffer().load_file("samples/birds.wav"),
sc.Buffer().load_file("samples/rain.wav"),
sc.Buffer().load_file("samples/water-flow.wav"),
sc.Buffer().load_file("samples/seaguls.wav"),
sc.Buffer().load_file("samples/motor.wav"),
sc.Buffer().load_file("samples/horse.wav"),
sc.Buffer().load_file("samples/wind.wav"),
sc.Buffer().load_file("samples/bell.wav"),
sc.Buffer().load_file("samples/guzheng.wav"),
sc.Buffer().load_file("samples/bamboo.wav"),
sc.Buffer().load_file("samples/pipa.wav"),
sc.Buffer().load_file("samples/taiko.wav"),
sc.Buffer().load_file("samples/sheeps.wav"),
sc.Buffer().load_file("samples/thunder.wav"),
sc.Buffer().load_file("samples/bellJapan.wav")   
]

birds = buffers[0].bufnum 
rain = buffers[1].bufnum
water = buffers[2].bufnum
seaguls = buffers[3].bufnum
motor = buffers[4].bufnum
horse = buffers[5].bufnum
wind = buffers[6].bufnum
bell = buffers[7].bufnum
guzheng = buffers[8].bufnum
bamboo = buffers[9].bufnum
pipa = buffers[10].bufnum
taiko = buffers[11].bufnum
sheeps = buffers[12].bufnum
thunder = buffers[13].bufnum
bellJapan = buffers[14].bufnum

bufnums = [
    [water, seaguls, motor],
    [water, birds, horse],
    [water, birds, rain],
    [],
    [],
    [],
    [water],
    [birds],
    [water, birds, rain]
]


/Users/sasan/anaconda3/lib/python3.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [6]:
%%sc // SynthDefs 
SynthDef("pb-simple", { 
    arg out=0, bufnum=0, rate=1, pan=0, amp=0.3, loop=1, 
    lgrt=2, lgamp = 0.5, cf=1000, rq=1, mix=0, room=0, damp=0.5; 
    
    var sig;
    
    sig = PlayBuf.ar(2, bufnum, rate.lag(lgrt)*BufRateScale.kr(bufnum), loop: loop, doneAction: 2);
    sig = BPF.ar(sig, cf, rq);
    sig = FreeVerb2.ar(sig, sig, mix, room, damp);
    
    Out.ar(out, Pan2.ar(sig, pan, amp.lag(lgamp)));
}).add;

SynthDef("pb-env", { 
    arg out=0, bufnum=0, rate=1, pan=0, amp=0.3, loop=1, 
    lgrt=2, lgamp = 0.5, cf=1000, rq=1, mix=0, room=0, damp=0.5,
    atk=2, sus=0, rel=3, c1=1, c2=(-1); 
    
    var sig, env;
    
    env = EnvGen.kr(Env([0,1,1,0],[atk,sus,rel],[c1,0,c2]),doneAction:2);
    sig = PlayBuf.ar(2, bufnum, rate.lag(lgrt)*BufRateScale.kr(bufnum), loop: loop, doneAction: 2);
    sig = BPF.ar(sig, cf, rq);
    sig = FreeVerb2.ar(sig, sig, mix, room, damp);
    sig = sig * env;
    
    Out.ar(out, Pan2.ar(sig, pan, amp.lag(lgamp)));
}).add;

SynthDef("fm", {
    arg freq=500, mRatio=1, cRatio=1, index=1, iScale=5, 
    cAtk=4, cRel=(-4), amp=0.2, atk=0.01, rel=3, 
    pan=0, out=0, fx=0, fxsend=(-25);
    
    var car, mod, env, iEnv;

    iEnv = EnvGen.kr(
        Env(
            [index, index*iScale, index],
            [atk, rel],
            [cAtk, cRel]
        )
    );

    env = EnvGen.kr(Env.perc(atk,rel,curve:[cAtk,cRel]),doneAction:2);
    mod = SinOsc.ar(freq * mRatio, mul:freq * mRatio * iEnv);
    car = SinOsc.ar(freq * cRatio + mod) * env * amp;
    car = Pan2.ar(car, pan);

    Out.ar(out, car);
}).add;

SynthDef("bell", {
    |fs=1, t60=1, pitchy=1, amp=0.25, gate=1|
    var sig, exciter;
    //exciter = Impulse.ar(0);
    exciter = WhiteNoise.ar() * EnvGen.ar(Env.perc(0.001, 0.05), gate) * 0.25;
    sig = Klank.ar(
        `[
            [1, 2, 2.803, 3.871, 5.074, 7.81, 10.948, 14.421],   // freqs
            [1, 0.044, 0.891, 0.0891, 0.794, 0.1, 0.281, 0.079], // amplitudes
            [1, 0.205, 1, 0.196, 0.339, 0.047, 0.058, 0.047]*t60     // ring times
        ],
        exciter,
        freqscale:fs*pitchy);
    sig = FreeVerb.ar(sig) * amp;
    DetectSilence.ar(sig, 0.001, 0.5, doneAction:2);
    Out.ar(0, sig!2);
}).add;


SynthDef(\bpfsaw, {
    arg atk=2, sus=0, rel=3, c1=1, c2=(-1),
    freq=500, detune=0.2, pan=0, cfhzmin=0.1, cfhzmax=0.3,
    cfmin=500, cfmax=2000, rqmin=0.1, rqmax=0.2,
    lsf=200, ldb=0, amp=1, out=0;
    
    var sig, env;
    
    env = EnvGen.kr(Env([0,1,1,0],[atk,sus,rel],[c1,0,c2]),doneAction:2);
    
    sig = Saw.ar(freq * {LFNoise1.kr(0.5,detune).midiratio}!2);
    sig = BPF.ar(
        sig,
        {LFNoise1.kr(
            LFNoise1.kr(4).exprange(cfhzmin,cfhzmax)
        ).exprange(cfmin,cfmax)}!2,
        {LFNoise1.kr(0.1).exprange(rqmin,rqmax)}!2
    );
    sig = BLowShelf.ar(sig, lsf, 0.5, ldb);
    sig = Balance2.ar(sig[0], sig[1], pan);
    sig = sig * env * amp;
    
    Out.ar(out, sig);
}).add;

SynthDef(\bpfsine, {
    arg atk=2, sus=0, rel=3, c1=1, c2=(-1),
    freq=500, detune=0.2, pan=0, cfhzmin=0.1, cfhzmax=0.3,
    cfmin=500, cfmax=2000, rqmin=0.1, rqmax=0.2,
    lsf=200, ldb=0, amp=1, out=0;
    var sig, env;
    env = EnvGen.kr(Env([0,1,1,0],[atk,sus,rel],[c1,0,c2]),doneAction:2);
    sig = SinOsc.ar(freq * {LFNoise1.kr(0.5,detune).midiratio}!2);
    sig = BPF.ar(
        sig,
        {LFNoise1.kr(
            LFNoise1.kr(4).exprange(cfhzmin,cfhzmax)
        ).exprange(cfmin,cfmax)}!2,
        {LFNoise1.kr(0.1).exprange(rqmin,rqmax)}!2
    );
    sig = BLowShelf.ar(sig, lsf, 0.5, ldb);
    sig = Balance2.ar(sig[0], sig[1], pan);
    sig = sig * env * amp;
    Out.ar(out, sig);
}).add;

SynthDef(\reverb, {
    arg in, predelay=0.1, revtime=1.8,
    lpf=4500, mix=0.15, amp=1, out=0;
    
    var dry, wet, temp, sig;
    
    dry = In.ar(in,2);
    temp = In.ar(in,2);
    wet = 0;
    temp = DelayN.ar(temp, 0,2, predelay);
    16.do{
        temp = AllpassN.ar(temp, 0.05, {Rand(0.001,0.05)}!2, revtime);
        temp = LPF.ar(temp, lpf);
        wet = wet + temp;
    };
    
    sig = XFade2.ar(dry, wet, mix*2-1, amp);
    
    Out.ar(out, sig);
}).add;

In [7]:
%%sc // assign and fill busses 
~bus = Dictionary.new;
~bus.add(\reverb -> Bus.audio(s,2));
~bus.add(\reverb_guzh -> Bus.audio(s,3));

In [8]:
%%sc // new reverb 

~out = 0;
~mainGroup = Group.new;
~reverbGroup = Group.after(~mainGroup);
~reverbSynth = Synth.new(\reverb, [
        \amp, 1,
        \predelay, 0.8,
        \revtime, 0.5,
        \lpf, 4500,
        \mix, 0.4,
        \in, ~bus[\reverb],
        \out, ~out,
    ], ~reverbGroup
);

~reverbGuzh = Synth.new(\reverb, [
        \amp, 0.8,
        \predelay, 1.5,
        \revtime, 1.5,
        \lpf, 5000,
        \mix, 0.5,
        \in, ~bus[\reverb_guzh],
        \out, ~out,
    ], ~reverbGroup
);


In [9]:
# japanese scales 
%sc g = [0,4,6,7,11,12].midiratio
%sc h = [0,1,5,6,10,12].midiratio
%sc i = [0,2,3,7,8,12].midiratio
%sc p = [0,2,4,6,8,10,12].midiratio
%sc ~gFreq = g * 220

In [10]:
%%sc // Pbinds and events

~paddur_min = 4.5;
~paddur_max = 5.5;
~mdur_min = 0.99;
~mdur_max = 1;
~mfreq = 1;
~mdetune = 0;
~mrq_min = 0.005;
~mrq_max = 0.008;
~mcf = 1;
~matk = 3;
~msus = 1;
~mrel = 5;
~mamp = 0.9;
~mpan_min = 0;
~mpan_max = 0;
~freq = 0;
~pasamp = 0;
~passcf = 1;

~gamp = 0;
~pamp = 0;
~bamp = 0;
~jamp = 0;
~acamp = 0;
~jpadamp = 0;
~guzhamp = 0;
~mjamp = 0;
~iSc = 1;
~durScale = 1;

e = Dictionary.new;

e.add(\pad_sine_lf -> {
    ~chords = Pbind(
        \instrument, \bpfsine,
        \dur, Pwhite(Pfunc{~paddur_min}, Pfunc{~paddur_max}),
        \midinote, Pxrand([
            [23,35,54,63,64],
            [45,52,54,59,61,64],
            [28,40,47,56,59,63],
            [42,52,57,61,63]
        ], inf),
        \detune, Pexprand(0.05,0.2),
        \cfmin, 500,
        \cfmax, 1000,
        \rqmin, Pexprand(0.01,0.02),
        \rqmax, Pexprand(0.2,0.3),
        \atk, Pwhite(2.0,2.5),
        \rel, Pwhite(6.5,10.0),
        \ldb, 6,
        \amp, 0.2,
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).play;
    
    ~marimba_one = Pbind(
        \instrument, \bpfsaw,
        \dur, Pwhite(Pfunc{~mdur_min}, Pfunc{~mdur_max}),
        \freq, Prand([1/2, 2/3, 1], inf) * Pfunc{~mfreq},
        \detune, Pfunc({~mdetune}),
        \rqmin, Pfunc{~mrq_min},
        \rqmax, Pfunc{~mrq_max},
        \cfmin, Prand((Scale.major.degrees+64).midicps,inf) *
        (Prand(([1,2,4]), inf) * round((Pfunc{~mcf}))),
        \cfmax, Pkey(\cfmin) * Pwhite(1.008,1.025),
        \atk, Pfunc{~matk},
        \sus, Pfunc{~msus},
        \rel, Pfunc{~mrel},
        \amp, Pfunc{~mamp},
        \pan, Pwhite(Pfunc{~mpan_min},Pfunc{~mpan_max}),
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).collect({ |event|
    ~marimba_oneEvent = event;
}).play;
});

e.add(\passage_one -> {
~passage_one = Pbind(\instrument, "bell",
    \dur, Pwhite(0.1,0.25, 1),
    \fs, Prand((Scale.major.degrees+64).midicps,inf),
    \amp, Pwhite(0.001,0.003),
    \t60, 9,
    \pitchy, 0.5 + Pfunc{~passcf},
    \group, ~mainGroup,
    \out, 0
).play;
});


e.add(\passage_two -> {
~passage_two = Pbind(\instrument, "bell",
    \dur, Pwhite(0.1,0.25, 3),
    \fs, Prand((Scale.major.degrees+64).midicps,inf),
    \amp, Pwhite(0.001,0.003),
    \t60, 9,
    \pitchy, 0.5 + Pfunc{~passcf},
    \group, ~mainGroup,
    \out, 0
).play;
});

e.add(\passage_three -> {
~passage_three = Pbind(\instrument, "bell",
    \dur, Pwhite(0.05,0.15, 5),
    \fs, Prand((Scale.major.degrees+64).midicps,inf),
    \amp, Pwhite(0.001,0.003),
    \t60, 9,
    \pitchy, 0.5 + Pfunc{~passcf},
    \group, ~mainGroup,
    \out, 0
).play;
});


e.add(\melody_high -> {
~guzh = Pbind(\instrument, "pb-env",
    \bufnum, ^guzheng,
    \dur, Pxrand([0.05,0.1,0.2,0.25,0.5,0.666,0.75,1,1.5,2,Rest(1),Rest(0.5),Rest(0.75),Rest(0.25)], inf),
    \rate, Prand(g, inf) * 1.005,
    \atk, 0.01, 
    \sus, 0, 
    \rel, 5,
    \loop, 0,
    \amp, Pfunc{~gamp},     
    \out, ~bus[\reverb_guzh],
).collect({ |event|
    ~lastGuzhEvent = event;
}).play;
});

e.add(\melody_low -> {
~pip = Pbind(\instrument, "pb-env",
    \bufnum, ^pipa,
    \dur, Pfunc{~lastGuzhEvent[\dur]},
    \rate, Prand(g, inf) * 0.5,
    \atk, 0.01, 
    \sus, 0, 
    \rel, 5,
    \cf, 500,
    \loop, 0,
    \amp, Pfunc{~pamp},   
    \out, ~bus[\reverb_guzh],
).play;
});

e.add(\low_pad -> {
~bamb = Pbind(\instrument, "pb-env",
    \bufnum, ^bamboo,
    \dur, Prand([0.1,0.2,0.5,2,4,6], inf),
    \atk, 2, 
    \sus, 1, 
    \rel, 3,
    \rate, Prand(g, inf) * 0.35,
    \loop, 0,
    \amp, Pfunc{~bamp},   
    \out, ~bus[\reverb_guzh],
).play;
});

e.add(\japan_melody -> {
~japanMelody = Pbind(\instrument, "fm",
    \dur, Pwhite(0.55, 0.66) / Pfunc{~durScale},
    \freq, Pxrand(~gFreq, inf),
    \mRatio, 1,
    \cRatio, 1,
    \index, 1,
    \iScale, Pfunc{~iSc},
    \atk, 0,
    \rel, Pexprand(0.4,0.6, inf),
    \amp, Pwhite(1, 2) * Pfunc{~jamp},
    \pan, Pseq([-0.25,0.25],inf),
    \out, ~bus[\reverb_guzh]
).play 
});

e.add(\japan_accomp -> {
~japanAccomp = Pbind(\instrument, "pb-env",
    \bufnum, ^pipa,
    \dur, Pseq([0.666, 0.666, 0.666, 1.998], inf) / Pfunc{~durScale},
    \rate, Pseq([g[0],g[1],g[2],g[3],g[4],g[5],g[4],g[3],g[4],g[5],g[4],g[2],g[4],g[5],g[4],g[1]], inf) * 0.5,
    \atk, 0, 
    \sus, 0, 
    \rel, 5,
    \cf, 500,
    \loop, 0,
    \amp, Pwhite(1, 2) * Pfunc{~acamp},   
    \out, 0,
).play 
});

e.add(\j_pad -> {
~jPad = Pbind(\instrument, "fm",
    \dur, Pwhite(2, 4),
    \freq, Pseq([[~gFreq[0],~gFreq[2]],[~gFreq[1],~gFreq[3]],[~gFreq[2],~gFreq[4]],[~gFreq[3],~gFreq[5]]], inf),
    \mRatio, 1,
    \cRatio, 1,
    \index, 0.5,
    \iScale, 3,
    \atk, 2,
    \rel, 2,
    \amp, Pfunc{~jpadamp},
    \pan, Pseq([-0.25,0.25],inf),
    \out, ~bus[\reverb_guzh]
).play 
});

e.add(\guzheng_melody -> {
~guzhMelody = Pbind(\instrument, "pb-env",
    \bufnum, ^guzheng,
    \dur, Pwhite(0.55, 0.66) / Pfunc{~durScale},
    \rate, Pxrand(g, inf) * 1.005,
    \atk, 0.5,
    \rel, 5,
    \loop, 0,
    \amp, Pwhite(1, 1.5) * Pfunc{~guzhamp},
    \pan, Pseq([-0.25,0.25],inf),
    \out, ~bus[\reverb_guzh]
).play 
});

e.add(\japan_marimba -> {
~marimba_two = Pbind(
        \instrument, "bpfsaw",
        \dur, Prand([0.5,1], inf),
        \freq, Prand([1/2, 2/3, 1], inf),
        \detune, Pwhite(0,1),
        \rqmin, 0.005,
        \rqmax, 0.008,
        \cfmin, Prand(~gFreq, inf) * Prand([2,4,8],inf),
        \cfmax, Pkey(\cfmin) * Pwhite(1.008,1.025),
        \atk, 3,
        \sus, 1,
        \rel, 5,
        \amp, Pfunc{~mjamp},
        \pan, Pwhite(-1,1),
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).play;
});

e.add(\trill -> {
~guzhTrill = Pbind(\instrument, "pb-env",
    \bufnum, ^guzheng,
    \dur, 0.05,
    \rate, g[0] * 1.005,
    \loop, 0,
    \amp, 0.5,     
    \out, ~bus[\reverb],
).play;
});


e.add(\stop -> {
    ~guzh.stop;
    ~pip.stop;
    ~bamb.stop;
    ~japanMelody.stop;
    ~japanAccomp.stop;
    ~jPad.stop;
    ~guzhMelody.stop;
    ~marimba_one.stop;
    ~chords.stop;
    ~marimba_two.stop;
});


## Thread for Playing Data

In [11]:
class Bloodplayer:
    
    def __init__(self, data, pulse_time=0.1, verbose=False):
        self.lock = threading.Lock()
        self.stopevent = threading.Event()
        self.callback_fn = None
        self.idx = 0
        self.data = data
        self.length = data.shape[0]
        self.verbose = verbose
        self.pulse_time = pulse_time
        self.rtime = 0
    
    def callback_fn_default(self, v):
        os.write(1, f"\r                       \r{v}".encode())
        
    def procfn(self):
        self.idx = 0
        self.rtime = 0
        while not self.stopevent.wait(0) and self.idx < self.length-1:
            v = self.data[self.idx]
            if self.verbose: 
                os.write(1, f"\r{self.idx}:{self.idx}                   ".encode())
            if callable(self.callback_fn):
                self.callback_fn(self)
            else:
                self.callback_fn_default(v)
            self.rtime += self.pulse_time
            self.idx = int(self.rtime)
            time.sleep(self.pulse_time)
        #print("done.")
    
    def set_callback(self, fn):
        self.callback_fn = fn
        
    def create_thread(self):
        threadname = "BloodPlayer-thread"
        # check first if it already exists
        if threadname in [t.name for t in threading.enumerate()]:
            print("create_thread: thread is already existing, stop first")
        else:
            self.stopevent.clear()
            self.producer = threading.Thread(name=threadname, target=self.procfn, args=[])
            self.producer.start()

    def stop_thread(self):
        self.stopevent.set()

In [12]:
bloodplayer = Bloodplayer(delta)

## Plot Data

In [13]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [14]:
# create figure

def create_figure():
    global fig, ax, mngr, plmarked, prev_idx 
    global legend_nature, legend_algomus, legend_japan
    
    fig, ax = plt.subplots(1) 
    mngr = plt.get_current_fig_manager(); 
    mngr.window.setGeometry(465, 209, 975, 611)
    ax.clear()
    plt.xlim(-0.1, len(index))
    plt.ylim(-0.1, delta_max+1)
    plmarked, = ax.plot([], [], "r-", lw=1)
    plmarked, = ax.plot([], [], "g-", lw=1)
    plmarked, = ax.plot([], [], "y-", lw=1)
    plmarked, = ax.plot([], [], "C9-", lw=1)
    plmarked, = ax.plot([], [], "r-", lw=1)
    prev_idx = 0
    ax.set_xlabel('time in seconds', color='grey')
    ax.set_ylabel('signal value', color='grey')
    ax.set_title(r'Sonification for Process Monitoring in Highly Sensitive Situations')
    legend_nature = plt.legend(('moving average 5s', 'moving average 30s', 'moving average 120s', '0.75 threshold'))
    legend_algomus = plt.legend(('delta', 'filtered delta'))
    legend_japan = plt.legend(('filtered delta', 'moving average 30s', 'moving average 120s'))
    connection_id = fig.canvas.mpl_connect('button_press_event', onclick)

def update_plot_nature(self, t, delta_val, volume_val, xs, vs0, vs1, vs2, ts=None, ys=None):
    global fig, ax, plmarked, plv0, plv1, plv2
    global pldata, filter_plot, prev_idx

    plmarked.set_data([t,t], [-10, 10])
    thrsh.set_data([-10,1000], [0.7, 0.7])
    plv0.set_xdata(xs[0:])
    plv0.set_ydata(vs0[0:])
    plv1.set_xdata(xs[0:])
    plv1.set_ydata(vs1[0:])
    plv2.set_xdata(xs[0:])
    plv2.set_ydata(vs2[0:])

    ax.draw_artist(ax.patch)
    ax.draw_artist(thrsh)
    ax.draw_artist(plv0)
    ax.draw_artist(plv1)
    ax.draw_artist(plv2)
    ax.draw_artist(plmarked)
    progress.value = volume_val
    ax.draw_artist(legend_nature)
    fig.canvas.update()

def update_plot_algomus(self, t, delta_val, volume_val, xs, vs0, vs1, vs2, ts=None, ys=None):
    global fig, ax, plmarked, plv0, plv1, plv2
    global pldata, filter_plot, prev_idx

    if not (prev_idx == self.idx):
        filter_plot.set_xdata(ts[0:])
        filter_plot.set_ydata(ys[0:])
        prev_idx = self.idx
    progress.value = volume_val
    pldata.set_xdata(xs[0:])
    pldata.set_ydata(delta_val)
    plmarked.set_data([t,t], [-10, 10])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(filter_plot)
    ax.draw_artist(plmarked)
    ax.draw_artist(legend_algomus)
    fig.canvas.update()

def update_plot_japan(self, t, delta_val, volume_val, xs, vs0, vs1, vs2, ts=None, ys=None):
    global fig, ax, plmarked, plv0, plv1, plv2
    global pldata, filter_plot, prev_idx

    if not (prev_idx == self.idx):
        filter_plot.set_xdata(ts[0:])
        filter_plot.set_ydata(ys[0:])
        prev_idx = self.idx
    progress.value = volume_val
    plv1.set_xdata(xs[0:])
    plv1.set_ydata(vs1[0:])
    plv2.set_xdata(xs[0:])
    plv2.set_ydata(vs2[0:])
    plmarked.set_data([t,t], [-10, 10])
    ax.draw_artist(ax.patch)
    ax.draw_artist(plv1)
    ax.draw_artist(plv2)
    ax.draw_artist(filter_plot)
    ax.draw_artist(plmarked)
    ax.draw_artist(legend_japan)
    fig.canvas.update()

def onclick(event):
    global bloodplayer
    if event.dblclick:
        print(event.button, event.xdata)
        bloodplayer.rtime = int(event.xdata)

create_figure()
connection_id = fig.canvas.mpl_connect('button_press_event', onclick)


## Sonification

In [15]:
# Custom code for sonifications

v0 = [0,0]
v1 = [0,0]
v2 = [0,0]
vs0 = []
vs1 = []
vs2 = []
deltas = []
xs = []
tau = [5, 30, 120]

order = 1
cfs = [0.01, 0.05, 0.5]
sr = 1/bloodplayer.pulse_time
b = []
a = []
zi = []
zl = []
z = [None] * len(cfs)
ts = []
ys = []

takt = [0,0]
amp = [0.0,0.0,0.0]
amp_j = [0.0,0.0,0.0]
rate = [0,0,0]
pan = [0,0,0]
cf = [0,0,0]
takt_rate = 50 # each 50 ml
volume_val = 0
node_base_cont = 5000
node_base_event = 5010
node_base_clock = 5050

major = [64, 66, 68, 69, 71, 73, 75, 76, 78, 80, 81, 83, 85, 87, 88]


for k, cf in enumerate(cfs):
    bk, ak = signal.butter(order, cf, fs=sr)
    b.append(bk)
    a.append(ak)
    zi.append(signal.lfilter_zi(bk, ak))
    zl.append([0, 0, 0])

# initiall play buffers for the selected sonification
def nature_init(bufnums):
    if not bufnums:
        os.write(1, "no buffer needed to initiate for this sonification   ".encode())
    else:
        for i, bufnum in enumerate(bufnums):
            sc.msg("/s_new", ["pb-simple", (node_base_cont + i), 1, 1, "bufnum", bufnum, "rate", 1, "amp", 0])

def tau_zero(self, volume_val):
    global amp
    refidx = max(self.idx-tau[0], 0)
    v0[0] = (volume.values[self.idx] - volume.values[refidx])/tau[0]
    amp[0] = scn.linlin(v0[0], 0, 3, 0.2, 1)
    sc.msg("/n_set", [node_base_cont, "rate", 1, "amp", amp[0], "lgrt", 3, "lgamp", 1])
    if v0[0] >= 1.5:
        rate[0] = scn.linlin(v0[0], 1.5, 3.2, 1.5, 4)
        sc.msg("/n_set", [node_base_cont, "rate", rate[0]])

def event_off(buf_node, bufnum):
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", bufnum, "rate", 0.5, "amp", 0.4, "loop", 1, "lgrt", 2, "lgamp", 2, "cf", 400])
    time.sleep(2)
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", bufnum, "rate", 0.3, "amp", 0.2, "loop", 1, "lgrt", 2, "lgamp", 3, "cf", 200])
    time.sleep(3)
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", bufnum, "rate", 0.2, "amp", 0.05, "loop", 0, "lgrt", 1, "lgamp", 1, "cf", 100])
    time.sleep(1)
    sc.msg("/n_free", [buf_node])

def clock_event(bufnum, takt, rate=1, amp=0.1, cf=1000, rq=1, mix=0, room=0, damp=0.5, out=0):  
    for t in range(1, takt+1): 
        sc.msg("/s_new", ["pb-simple", (node_base_clock + t), 1, 1, "bufnum", bufnum, "rate", rate, "amp", amp, "loop", 0, "cf", cf, "rq", rq, "mix", mix, "room", room, "out", out])
        time.sleep(1)
    #print("event clock: " + str(takt))

def plot_and_update(self, delta_val, volume_val, v0, v1, v2, zl=None, *filters):
    if not filters:
        xs.append(self.idx)
        vs0.append(v0[0])
        vs1.append(v1[0])
        vs2.append(v2[0])
        deltas.append(delta_val)
        update_plot(self, self.idx, deltas, volume_val, xs, vs0, vs1, vs2)        
    else:
        xs.append(self.idx)
        vs0.append(v0[0])
        vs1.append(v1[0])
        vs2.append(v2[0])
        ts.append(self.rtime)
        ys.append((zl))
        deltas.append(delta_val)
        update_plot(self, self.rtime, deltas, volume_val, xs, vs0, vs1, vs2, ts, ys, True)

def plot_and_update_nature(self, delta_val, volume_val, v0, v1, v2, zl=None):
    xs.append(self.idx)
    vs0.append(v0[0])
    vs1.append(v1[0])
    vs2.append(v2[0])
    deltas.append(delta_val)
    update_plot_nature(self, self.idx, deltas, volume_val, xs, vs0, vs1, vs2)        

def plot_and_update_algomus(self, delta_val, volume_val, v0, v1, v2, zl=None):
    xs.append(self.idx)
    vs0.append(v0[0])
    vs1.append(v1[0])
    vs2.append(v2[0])
    ts.append(self.rtime)
    ys.append((zl))
    deltas.append(delta_val)
    update_plot_algomus(self, self.rtime, deltas, volume_val, xs, vs0, vs1, vs2, ts, ys)

def plot_and_update_japan(self, delta_val, volume_val, v0, v1, v2, zl=None):
    xs.append(self.idx)
    vs0.append(v0[0])
    vs1.append(v1[0])
    vs2.append(v2[0])
    ts.append(self.rtime)
    ys.append((zl))
    deltas.append(delta_val)
    update_plot_japan(self, self.rtime, deltas, volume_val, xs, vs0, vs1, vs2, ts, ys)

def sonification_nature_one(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp, rate, pan, cf, takt_rate

    # assign volume
    volume_val = volume[self.idx]
    
    # *** water *** tau 0 = 5 seconds
    tau_zero(self, volume_val)
        
    # *** seaguls *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    if v1[0] < 0.5:
        amp[1] = v1[0]/2
        sc.msg("/n_set", [node_base_cont + 1, "rate", 1, "amp", amp[1]])

    if v1[0] >= 0.5:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1)
        rate[1] = scn.linlin(v1[0], 0.5, 2, 1, 2)
        sc.msg("/n_set", [node_base_cont + 1, "rate", rate[1], "amp", amp[1]])
        
        # *** thunder *** event for exceeding threshold! 
        if v1[1] < 0.7 and v1[0] >= 0.7:
            sc.msg("/s_new", ["pb-simple", node_base_event, 1, 1, "bufnum", thunder, "rate", 0.9, "amp", 0.9, "loop", 1, "lgrt", 2, "lgamp", 2, "cf", 500])
        if v1[0] < 0.7 and v1[1] >= 0.7:
            event_off(node_base_event, thunder)
    
    v1[1] = v1[0]        

    # *** motor *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] >= 0.3:
        amp[2] = scn.linlin(v2[0], 0.3, 0.5, 0.01, 0.07)
        rate[2] = scn.linlin(v2[0], 0.3, 0.5, 0.5, 0.8)
        pan[2] = scn.linlin(v2[0], 0.3, 0.5, -1, 0)
        sc.msg("/n_set", [node_base_cont + 2, "rate", rate[2], "amp", amp[2], "pan", pan[2], "cf", 700, "mix", 0.2, "room", 0.2, "damp", 0.5])        

    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())

    # plot data
    plot_and_update_nature(self, delta[self.idx], volume_val, v0, v1, v2)

    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(bell, takt[0])
    takt[1] = takt[0]

def sonification_nature_two(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp, rate, pan, cf, takt_rate, volume_val
    
    # assign volume
    volume_val = volume[self.idx]
    
    # *** water *** tau 0 = 5 seconds
    tau_zero(self, volume_val)

    # *** birds *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    if v1[0] < 0.5:
        amp[1] = v1[0]/2
        sc.msg("/n_set", [node_base_cont + 1, "rate", 1, "amp", amp[1]])

    if v1[0] >= 0.5:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1)
        rate[1] = scn.linlin(v1[0], 0.5, 2, 1, 2.5)
        sc.msg("/n_set", [node_base_cont + 1, "rate", rate[1], "amp", amp[1]])
        
        # *** sheeps *** event for exceeding threshold!
        if v1[1] < 0.7 and v1[0] >= 0.7:
            sc.msg("/s_new", ["pb-simple", node_base_event, 1, 1, "bufnum", sheeps, "rate", 1, "amp", 0.6, "loop", 1])
        if v1[0] < 0.7 and v1[1] >= 0.7:
            event_off(node_base_event, sheeps)

    v1[1] = v1[0]        

    # *** horse *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] >= 0.25:
        amp[2] = scn.linlin(v2[0], 0.25, 0.5, 0.2, 0.8)
        rate[2] = scn.linlin(v2[0], 0.25, 0.5, 0.6, 1.2)
        pan[2] = scn.linlin(v2[0], 0.25, 0.5, 1, 0)

        sc.msg("/n_set", [node_base_cont + 2, "rate", rate[2], "amp", amp[2], "pan", pan[2]])        

    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())
    
    # plot data
    plot_and_update_nature(self, delta[self.idx], volume_val, v0, v1, v2)

    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(bell, takt[0])
    takt[1] = takt[0]
    
def sonification_nature_three(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp, rate, pan, cf, takt_rate
    
    # assign volume
    volume_val = volume[self.idx]

    # *** water *** tau 0 = 5 seconds
    tau_zero(self, volume_val)

    # *** birds *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    if v1[0] < 0.5:
        amp[1] = v1[0]/2
        sc.msg("/n_set", [node_base_cont + 1, "rate", 1, "amp", amp[1]])

    if v1[0] >= 0.5:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1)
        rate[1] = scn.linlin(v1[0], 0.5, 2, 1, 2.5)
        sc.msg("/n_set", [node_base_cont + 1, "rate", rate[1], "amp", amp[1]])

        # *** thunder *** event for exceeding threshold! 
        if v1[1] < 0.7 and v1[0] >= 0.7:
            sc.msg("/s_new", ["pb-simple", node_base_event, 1, 1, "bufnum", thunder, "rate", 0.9, "amp", 0.9, "loop", 1, "lgrt", 2, "lgamp", 2, "cf", 500])
        if v1[0] < 0.7 and v1[1] >= 0.7:
            event_off(node_base_event, thunder)
    
    v1[1] = v1[0]        

    # *** rain *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] >= 0.25:
        amp[2] = scn.linlin(v2[0], 0.25, 0.5, 0.1, 0.6)
        rate[2] = scn.linlin(v2[0], 0.25, 0.5, 0.6, 1.2)
        pan[2] = scn.linlin(v2[0], 0.25, 0.5, 1, 0)
        #cf[2] = scn.linlin(v2[0], 0.25, 0.5, 500, 2000)

        #sc.msg("/n_set", [node_base_cont + 2, "rate", rate[2], "amp", amp[2], "pan", pan[2], "cf", cf[2]])        
        sc.msg("/n_set", [node_base_cont + 2, "rate", rate[2], "amp", amp[2], "pan", pan[2]])        

    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())
    
    # plot data
    plot_and_update_nature(self, delta[self.idx], volume_val, v0, v1, v2)
    
    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(bell, takt[0])
    takt[1] = takt[0]
    
def sonification_algomusic_one(self):
    global a, b, z, zi, zl, nd, nv, ts, ys
    global revtime, mix, predelay, amp 
    global paddur_min, paddur_max, mamp 
    global mdur_min, mdur_max, mfreq, mdetune, mrq_min, mrq_max, mcf, matk, msus, mrel, mpan_min, mpan_max 
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global rate, pan, cf, takt_rate, freq, pasamp, passcf
     
    
    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]
    
    # calculate filters
    for k, cf in enumerate(cfs):
        z[k], zi[k] = signal.lfilter(b[k], a[k], np.array([delta_val]), zi=zi[k])
        zl[k][0] = copy.copy(z[k][0])

    # delta filtered
    # max event
    if np.argmax(zl[1]) == 1:
        if zl[1][0] < 1:
            %sc ~pasamp = 0
        else:
            pasamp = scn.linlin(zl[1][0], 1, 3, 0, 0.01)
            passcf = scn.linlin(zl[1][0], 1, 3, 1, 4)
            
            %sc ~pasamp = ^pasamp
            %sc ~passcf = ^passcf
            %sc e[\passage_one].value;
            
    zl[1][2] = zl[1][1]
    zl[1][1] = zl[1][0]
         
        
    # normalize delta and volume
    nd = scn.linlin(delta_val, delta_min, delta_max, 0, 1)       
    nv = scn.linlin(volume_val, volume_min, volume_max, 0, 1)
    
    # set reverb based on normalized values of delta and volume
    revtime = scn.linlin(nv, 0, 1, 1.8, 0.5)
    mix = scn.linlin(nv, 0, 1, 0.5, 0.1)
    predelay = scn.linlin(nv, 0, 1, 0.4, 0.1)
    amp = scn.linlin(nv, 0, 1, 0.8, 0.2)        
    paddur_min = 4.5-(scn.linlin(nd,0,1,0,4))    
    paddur_max = 5.5-(scn.linlin(nd,0,1,0,4))
    mdur_min = scn.linlin(nd,0,1,0.99,0.05)
    mdur_max = scn.linlin(nd,0,1,1,0.1)
    mfreq = scn.linlin(nd,0,1,1,4)
    mdetune = scn.linlin(nd,0,1,0,2)
    mrq_min = scn.linlin(nv,0,1,0.005,0.09)
    mrq_max = scn.linlin(nv,0,1,0.008,0.2)
    mcf = scn.linlin(nd,0,1,1,5)
    matk = scn.linlin(nv,0,1,3,1.5)
    msus = scn.linlin(nv,0,1,1,0.5)
    mrel = scn.linlin(nv,0,1,5,2.5)
    mamp = scn.linlin(nd,0,0.1,0.05,0.7) 
    mpan_min = scn.linlin(nd,0,1,0,-1)
    mpan_max = scn.linlin(nd,0,1,0,1)

    %sc ~reverbSynth.set(\revtime, ^revtime, \mix, ^mix, \predelay, ^predelay, \amp, ^amp)        
    %sc ~paddur_min = ^paddur_min
    %sc ~paddur_max = ^paddur_max
    %sc ~mdur_min = ^mdur_min
    %sc ~mdur_max = ^mdur_max
    %sc ~mfreq = ^mfreq
    %sc ~mdetune = ^mdetune
    %sc ~mrq_min = ^mrq_min
    %sc ~mrq_max = ^mrq_max
    %sc ~mcf = ^mcf
    %sc ~matk = ^matk
    %sc ~msus = ^msus
    %sc ~mrel = ^mrel
    %sc ~mamp = ^mamp
    %sc ~mpan_min = ^mpan_min
    %sc ~mpan_max = ^mpan_max
    
    # plot data
    plot_and_update_algomus(self, delta[self.idx], volume_val, v0, v1, v2, zl[1][0])

def sonification_algomusic_two(self):
    global a, b, z, zi, zl, nd, nv, ts, ys
    global revtime, mix, predelay, amp 
    global paddur_min, paddur_max, mamp 
    global mdur_min, mdur_max, mfreq, mdetune, mrq_min, mrq_max, mcf, matk, msus, mrel, mpan_min, mpan_max 
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global rate, pan, cf, takt_rate, freq, pasamp, passcf
     
    
    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]
    
    # calculate filters
    for k, cf in enumerate(cfs):
        z[k], zi[k] = signal.lfilter(b[k], a[k], np.array([delta_val]), zi=zi[k])
        zl[k][0] = copy.copy(z[k][0])
    
    # delta filtered
    if np.argmax(zl[1]) == 1:
        if zl[1][0] < 1:
            %sc ~pasamp = 0
        else:
            pasamp = scn.linlin(zl[1][0], 1, 3, 0, 0.01)
            passcf = scn.linlin(zl[1][0], 1, 3, 1, 4)

            #%sc ~freq = ^freq
            %sc ~pasamp = ^pasamp
            %sc ~passcf = ^passcf
            %sc e[\passage_two].value;
        
    zl[1][2] = zl[1][1]
    zl[1][1] = zl[1][0]
    
    # normalize delta and volume
    nd = scn.linlin(delta_val, delta_min, delta_max, 0, 1)       
    nv = scn.linlin(volume_val, volume_min, volume_max, 0, 1)
    
    # set reverb based on normalized values of delta and volume
    revtime = scn.linlin(nv, 0, 1, 1.8, 0.5)
    mix = scn.linlin(nv, 0, 1, 0.5, 0.1)
    predelay = scn.linlin(nv, 0, 1, 0.4, 0.1)
    amp = scn.linlin(nv, 0, 1, 0.8, 0.2)        
    paddur_min = 4.5-(scn.linlin(nd,0,1,0,4))    
    paddur_max = 5.5-(scn.linlin(nd,0,1,0,4))
    mdur_min = scn.linlin(nd,0,1,0.99,0.1)
    mdur_max = scn.linlin(nd,0,1,1,0.2)
    mfreq = scn.linlin(nd,0,1,1,3)
    mdetune = scn.linlin(nd,0,1,0,15)
    mrq_min = scn.linlin(nv,0,1,0.005,0.09)
    mrq_max = scn.linlin(nv,0,1,0.008,0.2)
    mcf = scn.linlin(nd,0,1,1,3)
    matk = scn.linlin(nv,0,1,3,1.5)
    msus = scn.linlin(nv,0,1,1,0.5)
    mrel = scn.linlin(nv,0,1,5,2.5)
    mamp = scn.linlin(nd,0,0.1,0.05,0.7) 
    mpan_min = scn.linlin(nd,0,1,0,-1)
    mpan_max = scn.linlin(nd,0,1,0,1)

    %sc ~reverbSynth.set(\revtime, ^revtime, \mix, ^mix, \predelay, ^predelay, \amp, ^amp)        
    %sc ~paddur_min = ^paddur_min
    %sc ~paddur_max = ^paddur_max
    %sc ~mdur_min = ^mdur_min
    %sc ~mdur_max = ^mdur_max
    %sc ~mfreq = ^mfreq
    %sc ~mdetune = ^mdetune
    %sc ~mrq_min = ^mrq_min
    %sc ~mrq_max = ^mrq_max
    %sc ~mcf = ^mcf
    %sc ~matk = ^matk
    %sc ~msus = ^msus
    %sc ~mrel = ^mrel
    %sc ~mamp = ^mamp
    %sc ~mpan_min = ^mpan_min
    %sc ~mpan_max = ^mpan_max

    # plot data
    plot_and_update_algomus(self, delta[self.idx], volume_val, v0, v1, v2, zl[1][0])

def sonification_algomusic_three(self):
    global a, b, z, zi, zl, nd, nv, ts, ys
    global revtime, mix, predelay, amp 
    global paddur_min, paddur_max, mamp 
    global mdur_min, mdur_max, mfreq, mdetune, mrq_min, mrq_max, mcf, matk, msus, mrel, mpan_min, mpan_max 
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global rate, pan, cf, takt_rate, freq, pasamp, passcf
     
    
    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]
    
    # calculate filters
    for k, cf in enumerate(cfs):
        z[k], zi[k] = signal.lfilter(b[k], a[k], np.array([delta_val]), zi=zi[k])
        zl[k][0] = copy.copy(z[k][0])
    
    # delta filtered
    if np.argmax(zl[1]) == 1:
        if zl[1][0] < 1:
            %sc ~pasamp = 0
        else:
            pasamp = scn.linlin(zl[1][0], 1, 3, 0, 0.01)
            passcf = scn.linlin(zl[1][0], 1, 3, 0, 4)

            #%sc ~freq = ^freq
            %sc ~pasamp = ^pasamp
            %sc ~passcf = ^passcf
            %sc e[\passage_three].value;
        
    zl[1][2] = zl[1][1]
    zl[1][1] = zl[1][0]
    
    # normalize delta and volume
    nd = scn.linlin(delta_val, delta_min, delta_max, 0, 1)       
    nv = scn.linlin(volume_val, volume_min, volume_max, 0, 1)
    
    # set reverb based on normalized values of delta and volume
    revtime = scn.linlin(nv, 0, 1, 1.8, 0.5)
    mix = scn.linlin(nv, 0, 1, 0.5, 0.1)
    predelay = scn.linlin(nv, 0, 1, 0.4, 0.1)
    amp = scn.linlin(nv, 0, 1, 0.8, 0.2)        
    paddur_min = 4.5-(scn.linlin(nd,0,1,0,3))    
    paddur_max = 5.5-(scn.linlin(nd,0,1,0,3))
    mdur_min = scn.linlin(nd,0,1,0.99,0.2)
    mdur_max = scn.linlin(nd,0,1,1,0.3)
    mfreq = scn.linlin(nd,0,1,1,2)
    mdetune = scn.linlin(nd,0,1,0,15)
    mrq_min = scn.linlin(nv,0,1,0.005,0.09)
    mrq_max = scn.linlin(nv,0,1,0.008,0.2)
    mcf = scn.linlin(nd,0,1,1,3)
    matk = scn.linlin(nv,0,1,3,1.5)
    msus = scn.linlin(nv,0,1,1,0.5)
    mrel = scn.linlin(nv,0,1,5,2.5)
    mamp = scn.linlin(nd,0,0.1,0.05,0.7) 
    mpan_min = scn.linlin(nd,0,1,0,-1)
    mpan_max = scn.linlin(nd,0,1,0,1)

    %sc ~reverbSynth.set(\revtime, ^revtime, \mix, ^mix, \predelay, ^predelay, \amp, ^amp)        
    %sc ~paddur_min = ^paddur_min
    %sc ~paddur_max = ^paddur_max
    %sc ~mdur_min = ^mdur_min
    %sc ~mdur_max = ^mdur_max
    %sc ~mfreq = ^mfreq
    %sc ~mdetune = ^mdetune
    %sc ~mrq_min = ^mrq_min
    %sc ~mrq_max = ^mrq_max
    %sc ~mcf = ^mcf
    %sc ~matk = ^matk
    %sc ~msus = ^msus
    %sc ~mrel = ^mrel
    %sc ~mamp = ^mamp
    %sc ~mpan_min = ^mpan_min
    %sc ~mpan_max = ^mpan_max

    # plot data
    plot_and_update_algomus(self, delta[self.idx], volume_val, v0, v1, v2, zl[1][0])

def sonification_japan_one(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp_j, rate, pan, cf, takt_rate
    global a, b, z, zi, zl, nd, nv, ts, ys
    global gamp, pamp, bamp

    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]
    
    # calculate filters
    for k, cf in enumerate(cfs):
        z[k], zi[k] = signal.lfilter(b[k], a[k], np.array([delta_val]), zi=zi[k])
        zl[k][0] = copy.copy(z[k][0])
        
    # *** water *** delta-filtered
    amp_j[0] = scn.linlin(zl[1][0], 0, 3, 0.1, 1)
    rate[0] = scn.linlin(zl[1][0], 0, 3, 1, 3)
    sc.msg("/n_set", [node_base_cont, "rate", rate[0], "amp", amp_j[0], "lgrt", 3, "lgamp", 1])

    # *** guzheng & pipa *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    
    if v1[0] < 0.5:
        %sc ~gamp = 0
        %sc ~pamp = 0
    
    # guzheng
    elif v1[0] >= 0.5:
        gamp = scn.linlin(v1[0], 0.5, 3, 0.01, 0.5) 
        %sc ~gamp = ^gamp
     
    # pipa
    if v1[0] >= 0.75:
        pamp = scn.linlin(v1[0], 0.75, 3, 0.01, 0.3) 
        %sc ~pamp = ^pamp
    
    # *** bamboo pad *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] < 0.45:
        %sc ~bamp = 0
    else:
        bamp = scn.linlin(v2[0], 0.45, 1, 0.05, 0.2) 
        %sc ~bamp = ^bamp
    
    # plot data
    plot_and_update_japan(self, delta[self.idx], volume_val, v0, v1, v2, zl[1][0])
                    
    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(taiko, takt[0], rate=1, amp=0.2, mix=0)
    takt[1] = takt[0]
    
def sonification_japan_two(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp_j, rate, pan, cf, takt_rate
    global a, b, z, zi, zl, nd, nv, ts, ys
    global acamp, durScale, jamp, jpadamp, iSc

    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]
    
    # calculate filters
    for k, cf in enumerate(cfs):
        z[k], zi[k] = signal.lfilter(b[k], a[k], np.array([delta_val]), zi=zi[k])
        zl[k][0] = copy.copy(z[k][0])

    # *** birds *** delta-filtered
    amp_j[0] = scn.linlin(zl[1][0], 0, 3, 0.1, 0.9)
    rate[0] = scn.linlin(zl[1][0], 0, 3, 1, 2.5)
    sc.msg("/n_set", [node_base_cont, "rate", rate[0], "amp", amp_j[0], "lgrt", 3, "lgamp", 1])

    # *** japan melody *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    
    if v1[0] < 0.5:
        %sc ~jamp = 0
        %sc ~durScale = 1
        %sc ~iSc = 1
    
    # japan melody
    elif v1[0] >= 0.5:
        jamp = scn.linlin(v1[0], 0.5, 3, 0.01, 0.4)
        durScale = scn.linlin(v1[0], 0.5, 3, 1, 4)
        iSc = scn.linlin(v1[0], 0.5, 3, 1, 5)
        %sc ~jamp = ^jamp
        %sc ~durScale = ^durScale
        %sc ~iSc = ^iSc
    
    if v1[0] < 0.75:
        %sc ~acamp = 0

    # pipa
    elif v1[0] >= 0.75:
        acamp = scn.linlin(v1[0], 0.75, 3, 0.01, 0.2) 
        %sc ~acamp = ^acamp
    
    # *** fm pad *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] < 0.45:
        %sc ~jpadamp = 0
    else:
        jpadamp = scn.linlin(v2[0], 0.45, 1, 0.1, 0.2) 
        %sc ~jpadamp = ^jpadamp
    
    # plot data
    plot_and_update_japan(self, delta[self.idx], volume_val, v0, v1, v2, zl[1][0])
                    
    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(taiko, takt[0], rate=1, amp=0.2, mix=0)
    takt[1] = takt[0]
    
def sonification_japan_three(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp_j, rate, pan, cf, takt_rate
    global a, b, z, zi, zl, nd, nv, ts, ys
    global guzhamp, durScale, mjamp
    
    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]
    
    # calculate filters
    for k, cf in enumerate(cfs):
        z[k], zi[k] = signal.lfilter(b[k], a[k], np.array([delta_val]), zi=zi[k])
        zl[k][0] = copy.copy(z[k][0])

    
    # *** water & birds *** delta-filtered
    amp_j[0] = scn.linlin(zl[1][0], 0, 3, 0.1, 1)
    rate[0] = scn.linlin(zl[1][0], 0, 3, 1, 3)
    amp_j[1] = scn.linlin(zl[1][0], 0, 3, 0.05, 0.3)
    rate[1] = scn.linlin(zl[1][0], 0, 3, 1, 2)
    sc.msg("/n_set", [node_base_cont, "rate", rate[0], "amp", amp_j[0], "lgrt", 3, "lgamp", 1])
    sc.msg("/n_set", [node_base_cont + 1, "rate", rate[1], "amp", amp_j[1], "lgrt", 3, "lgamp", 1])

    
    # *** marimba japan & guzheng *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    
    if v1[0] < 0.5:
        %sc ~mjamp = 0

    # marimba japan
    elif v1[0] >= 0.5:
        mjamp = scn.linlin(v2[0], 0.5, 3, 0.6, 1) 
        %sc ~mjamp = ^mjamp

    # guzheng
    if v1[0] < 0.75:
        %sc ~guzhamp = 0
        %sc ~durScale = 1

    elif v1[0] >= 0.75:
        guzhamp = scn.linlin(v1[0], 0.75, 3, 0.01, 0.25) 
        durScale = scn.linlin(v1[0], 0.75, 3, 1, 4)
        %sc ~guzhamp = ^guzhamp
        %sc ~durScale = ^durScale

    # *** rain *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] < 0.45:
        amp_j[2] = 0
        rate[2] = 1
    else:
        amp_j[2] = scn.linlin(v1[0], 0.45, 1, 0, 0.7)
        rate[2] = scn.linlin(v1[0], 0.45, 1, 1, 2)
    
    sc.msg("/n_set", [node_base_cont + 2, "rate", rate[2], "amp", amp_j[2], "lgrt", 3, "lgamp", 1])
    
        

    # plot data
    plot_and_update_japan(self, delta[self.idx], volume_val, v0, v1, v2, zl[1][0])
                    
    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(bellJapan, takt[0], rate=0.5, amp=0.4, mix=0)
    takt[1] = takt[0]

def quit():
    sc.msg("/n_free", node_base_cont)
    sc.msg("/n_free", node_base_cont + 1)
    sc.msg("/n_free", node_base_cont + 2)
    sc.msg("/n_free", node_base_event)


## GUI

In [16]:
# GUI

selected_init = 0

# filling blood bar
progress = ipywidgets.FloatProgress(value=5, min=0, max=volume_max, description="volume 0-250ml", bar_style='danger',layout=Layout(width='20%', height='250px'))
progress.orientation = 'vertical'
display(progress)

def select_sonification(selection):
    global selected_init, thrsh, plv0, plv1, plv2, filter_plot, pldata, amp
    
    bloodplayer.set_callback(selection)
    if selection == sonification_nature_one:
        create_figure()
        amp = [0.0,0.0,0.0]
        bloodplayer.pulse_time = 1
        selected_init = 0
        plv0, = ax.plot([], [], "r-", lw=1)
        plv1, = ax.plot([], [], "g-", lw=1)
        plv2, = ax.plot([], [], "y-", lw=1)
        thrsh, = ax.plot([], [], "C9-", lw=1)

    if selection == sonification_nature_two:
        create_figure()
        amp = [0.0,0.0,0.0]
        bloodplayer.pulse_time = 1
        selected_init = 1
        plv0, = ax.plot([], [], "r-", lw=1)
        plv1, = ax.plot([], [], "g-", lw=1)
        plv2, = ax.plot([], [], "y-", lw=1)
        thrsh, = ax.plot([], [], "C9-", lw=1)
        
    if selection == sonification_nature_three:
        create_figure()
        amp = [0.0,0.0,0.0]
        bloodplayer.pulse_time = 1
        selected_init = 2
        plv0, = ax.plot([], [], "r-", lw=1)
        plv1, = ax.plot([], [], "g-", lw=1)
        plv2, = ax.plot([], [], "y-", lw=1)
        thrsh, = ax.plot([], [], "C9-", lw=1)
        
    if selection == sonification_algomusic_one:
        create_figure()
        bloodplayer.pulse_time = 0.1
        selected_init = 3
        filter_plot = plt.plot([], [], 'r-', lw=1)[0]
        pldata, = ax.plot([], [], "g-", lw=0.5)

    if selection == sonification_algomusic_two:
        create_figure()
        bloodplayer.pulse_time = 0.1
        selected_init = 4
        filter_plot = plt.plot([], [], 'r-', lw=1)[0]
        pldata, = ax.plot([], [], "g-", lw=0.5)

    if selection == sonification_algomusic_three:
        create_figure()
        bloodplayer.pulse_time = 0.1
        selected_init = 5
        filter_plot = plt.plot([], [], 'r-', lw=1)[0]
        pldata, = ax.plot([], [], "g-", lw=0.5)
        
    if selection == sonification_japan_one:
        create_figure()
        bloodplayer.pulse_time = 0.1
        selected_init = 6
        filter_plot = plt.plot([], [], 'r-', lw=1)[0]
        plv1, = ax.plot([], [], "g-", lw=1)
        plv2, = ax.plot([], [], "y-", lw=1)

    if selection == sonification_japan_two:
        create_figure()
        bloodplayer.pulse_time = 0.1
        selected_init = 7
        filter_plot = plt.plot([], [], 'r-', lw=1)[0]
        plv1, = ax.plot([], [], "g-", lw=1)
        plv2, = ax.plot([], [], "y-", lw=1)

    if selection == sonification_japan_three:
        create_figure()
        bloodplayer.pulse_time = 0.1
        selected_init = 8
        filter_plot = plt.plot([], [], 'r-', lw=1)[0]
        plv1, = ax.plot([], [], "g-", lw=1)
        plv2, = ax.plot([], [], "y-", lw=1)

interact(select_sonification, selection = {
    'nature_one': sonification_nature_one,
    'nature_two': sonification_nature_two,
    'nature_three': sonification_nature_three,
    'AlgoMusic_one': sonification_algomusic_one,
    'AlgoMusic_two': sonification_algomusic_two,
    'AlgoMusic_three': sonification_algomusic_three,
    'japan_one': sonification_japan_one,
    'japan_two': sonification_japan_two,
    'japan_three': sonification_japan_three,
})

def start(b):
    global bloodplayer
    bloodplayer.create_thread()
    nature_init(bufnums[selected_init])
    if selected_init == 3:
        %sc e[\pad_sine_lf].value;
        
    if selected_init == 4:
        %sc e[\pad_sine_lf].value;
        
    if selected_init == 5:
        %sc e[\pad_sine_lf].value;
        
    if selected_init == 6:
        %sc e[\melody_high].value;
        %sc e[\melody_low].value;
        %sc e[\low_pad].value;
        
    if selected_init == 7:
        %sc e[\japan_melody].value;
        %sc e[\japan_accomp].value;
        %sc e[\j_pad].value;
    
    if selected_init == 8:
        %sc e[\guzheng_melody].value;
        %sc e[\japan_marimba].value;
        
    os.write(2, f"event  \r{selected_init}  initiated         ".encode())
    
b1 = ipywidgets.Button(description='Start') 
b1.on_click(start)

def stop(b):
    global bloodplayer
    bloodplayer.stop_thread()
    quit()
    zl.append([0, 0, 0])
    if selected_init > 2:
        %sc e[\stop].value;
        
b2 = ipywidgets.Button(description='Stop') 
b2.on_click(stop)

out = ipywidgets.Output()
ipywidgets.HBox([b1, b2, out])

FloatProgress(value=5.0, bar_style='danger', description='volume 0-250ml', layout=Layout(height='250px', width…

interactive(children=(Dropdown(description='selection', options={'nature_one': <function sonification_nature_o…

1 157.46312655086845
1 226.26034739454096
